In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import pytorch_lightning as pl


In [2]:
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
])
dataset_train = torchvision.datasets.FashionMNIST('./data', train = True, download = True, transform = transform)
dataset_test = torchvision.datasets.FashionMNIST('./data', train = False, download = True, transform = transform)
dataloader_train = torch.utils.data.DataLoader(dataset_train, 256, shuffle = True, num_workers = 8)
dataloader_test = torch.utils.data.DataLoader(dataset_test, 256, shuffle = False, num_workers = 8)

In [3]:
class Mnist_classifier(nn.Module):
    def __init__(self, num_class = 10):
        super(Mnist_classifier, self).__init__()
        self.FeatureExtractor = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 3, 2, 1),
            nn.ReLU(True),
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.ReLU(True),
            nn.Conv2d(128, 256, 3, 2, 1),
            nn.ReLU(True),
            nn.Conv2d(256, 512, 3, 2, 1),
            nn.ReLU(True),
        )
        self.Classifier = nn.Sequential(
            nn.AdaptiveMaxPool2d(1),
            nn.Flatten(1),
            nn.Linear(512, num_class)
        )
    
    def forward(self, x):
        feature = self.FeatureExtractor(x)
        cls = self.Classifier(feature)
        return cls

In [4]:
class model_pl(pl.LightningModule):
    def __init__(self, num_class = 10):
        super(model_pl, self).__init__()
        self.model = Mnist_classifier(num_class)
    
    def forward(self, x):
        cls = self.model(x)
        return cls
    
    def configure_optimizers(self, lr = 2e-3):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
        x,y = train_batch
        cls = self.model(x)
        loss = nn.functional.cross_entropy(cls, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x,y = val_batch
        cls = self.model(x)
        loss = nn.functional.cross_entropy(cls, y)
        self.log('val_loss', loss)

In [5]:
model = model_pl(num_class = 10)

In [6]:
trainer = pl.Trainer(gpus = [0, 1, 2], strategy="dp", enable_checkpointing = False, max_epochs = 20, num_processes = 3)
trainer.fit(model, train_dataloaders = dataloader_train, val_dataloaders=dataloader_test)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name  | Type             | Params
-------------------------------------------
0 | model | Mnist_classifier | 1.6 M 
-------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.293     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Exception ignored in: <function _releaseLock at 0x7f72f26fdd30>
Traceback (most recent call last):
  File "/home/ddpm/anaconda3/lib/python3.8/logging/__init__.py", line 223, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 1262370, 1262498, 1262626, 1262754, 1262882, 1263010, 1263138) exited unexpectedly

In [ ]:
model = model.to('cpu')

In [ ]:
d = iter(dataloader_test)

In [ ]:
x,y = next(d)
y_ = model(x)
sum(y_.argmax(1) == y)/y.shape[0]